<a href="https://colab.research.google.com/github/EmadSoheili/DeepLearning-UofT-Canvas/blob/main/Checkpoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

%matplotlib inline

In [2]:
# Load data

attrition_df = pd.read_csv('Inputs/HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [3]:
# List categorical columns

attrition_cat = attrition_df.dtypes[attrition_df.dtypes == 'object'].index.tolist()
attrition_cat

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

In [4]:
# Count unique values in categorical columns

attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [5]:
# OneHotEncoder

enc = OneHotEncoder(sparse=False)
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [6]:
attrition_df = attrition_df.merge(encode_df, left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

<ipython-input-6-327fa53b9590>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  attrition_df = attrition_df.drop(attrition_cat,1)


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [7]:
# Split dataset into inputs and output

y = attrition_df['Attrition_Yes'].values
X = attrition_df.drop(['Attrition_Yes', 'Attrition_No'], 1).values

<ipython-input-7-da1ca898d9b6>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = attrition_df.drop(['Attrition_Yes', 'Attrition_No'], 1).values


In [8]:
# Split data into train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [9]:
# Scale data

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Create Depp Learning model

number_input_feature = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# Add hidden layers

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_feature, activation='relu'))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Add output layers

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 448       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Define the checkpoint path

os.makedirs('Checkpoints/', exist_ok=True)
checkpoint_path = 'Checkpoints/weights.{epoch:02d}.hdf5'

In [12]:
# Compile model

nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
# Create a callback for checkpoint

cp_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    verbose = 1,
    save_weights_only = True,
    save_freq = 'epoch')

In [14]:
# Train model

fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
29/35 [=======================>......] - ETA: 0s - loss: 0.6535 - accuracy: 0.6519
Epoch 1: saving model to Checkpoints/weights.01.hdf5
35/35 [==============================] - 2s 4ms/step - loss: 0.6472 - accuracy: 0.6570
Epoch 2/100
33/35 [===========================>..] - ETA: 0s - loss: 0.5592 - accuracy: 0.7973
Epoch 2: saving model to Checkpoints/weights.02.hdf5
35/35 [==============================] - 0s 6ms/step - loss: 0.5609 - accuracy: 0.7949
Epoch 3/100
30/35 [========================>.....] - ETA: 0s - loss: 0.5022 - accuracy: 0.8323
Epoch 3: saving model to Checkpoints/weights.03.hdf5
35/35 [==============================] - 0s 5ms/step - loss: 0.5060 - accuracy: 0.8212
Epoch 4/100
27/35 [======================>.......] - ETA: 0s - loss: 0.4834 - accuracy: 0.8148
Epoch 4: saving model to Checkpoints/weights.04.hdf5
35/35 [==============================] - 0s 5ms/step - loss: 0.4684 - accuracy: 0.8276
Epoch 5/100
30/35 [========================>.....] - ETA: 0s

In [15]:
# Evaluate model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

12/12 - 0s - loss: 0.3668 - accuracy: 0.8967 - 180ms/epoch - 15ms/step
Loss: 0.36683666706085205, Accuracy: 0.89673912525177


In [16]:
# Export Phase

# Define another model

number_input_feature = len(X_train_scaled[0])
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5
nn_new = tf.keras.models.Sequential()

# Hidden layers

nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer

nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [17]:
# Compile the model

nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Restore the model weights

nn_new.load_weights('Checkpoints/weights.100.hdf5')

In [19]:
# Evaluate model

model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.3668 - accuracy: 0.8967 - 248ms/epoch - 21ms/step
Loss: 0.36683666706085205, Accuracy: 0.89673912525177


In [20]:
# Save the full model

nn_new.save('Saved_model/trained_model2.h5')

In [21]:
# Import full model

nn_imported = tf.keras.models.load_model('Saved_model/trained_model2.h5')

In [22]:
# Evaluate the completed model using the test data

model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.3668 - accuracy: 0.8967 - 146ms/epoch - 12ms/step
Loss: 0.36683666706085205, Accuracy: 0.89673912525177
